In [7]:
%load_ext autoreload
%autoreload 2

#hsvs includes
from hsvs.tools import dataset, framed_audio, magnitude, synthesis
import hsvs

import soundfile
import sounddevice

# 3rd party dependencies
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
vowel  = 'a'  # [a,e,i,o,u]
singer = 'm4' # [f1 - f9, m1-m11]

# collecting sorce / output file paths 
singer_vowel_dir = singer + '_' + vowel
source_file = dataset.get_sample('scales', 'slow_forte', vowel, singer)[0]
data_path = os.path.abspath(os.path.join(os.path.dirname(hsvs.__file__), os.pardir, 'data'))
json_file = os.path.join(data_path, 'results', singer_vowel_dir, 'audio.json' )
mat_file  = os.path.join(data_path, 'results', singer_vowel_dir, 'parameters.mat' )
out_file  = os.path.join(data_path, 'results', singer_vowel_dir, 'audio', 'synthesis_estimated.wav' )

# Load parameter trajectories from .mat file
parameters = sio.loadmat(mat_file)
pitch = parameters['f']
gain  = parameters['g']
Rd    = parameters['Rd']
zeros = parameters['z0']
poles = parameters['p0']
fs    = parameters['fs'][0,0]


# loading stored audio object
audio = framed_audio.FramedAudio.from_json(json_file, wav_replacement=source_file)

# run onset / offset analysis, add some margin to the values
onset, offset = magnitude.get_onset_offset(audio)
onset  += 50
offset -= 50

# prepare vectors for input (pitch) and output (overtone decibel magnitude)
overtones = audio.get_trajectory('overtones')[onset:offset, :]
pitch     = np.expand_dims(audio.get_trajectory('pitch')[onset:offset],1)

In [11]:
plt.plot(pitch)



sio.savemat('data/plots/frame.mat', {
    'pitch':pitch,          # pitch
    'gain':gain,            # gain
    'Rd':Rd,                # Rd parameter
    'zeros':zeros,          # filter zeros
    'poles': poles,         # filter poles
    'fs':fs,                # sample rate
    'harmonics':overtones}) 


NameError: name 'predictor' is not defined